In [ ]:
# NLP Lecture @ Strive School - 21st July 2021
# NER update

'''
Since today we are exploring the world of natural language processing, we’ll deepen in the Named Entity Recognition technique:
this is just one of the mechanisms that NLP embodies. The recognition of named entities as the process of automatic
identification of the entities present in a text and consequent classification into predefined categories such as "person",
"organization", "position" is a quite common activity and expect for English, trained models with spaCy offer few labels that
could be improved through training.

Following the case study of this morning, try to emulate it in order to label all the brands present in the provided datasets,
choosing the one you prefer OR trying to label all them and to train the model to recognize new different entities.
The result should be twofold: the final model should be able to recognize brands that it has already seen, but already new ones.
The brands proposed in the dataset concern fashion, cars and food.
In order to test the accuracy of the model, test it with sentences and brands the model has never seen.

Sample of the dataset
---------------------
- Cate Blanchett in Armani Privé. Rating: 8. Concludes as a rare butterfly, or from Rorschach's Test, or from computerized
axial tomography.
- I liked everything, recommend it! Another quality Xiaomi product...
- What is the price of that Fiat 500XL?

Info:
- Feel free to change or arrange a new dataset
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)

'''

In [1]:
# STEP 0 - PRE REQUISITES

# python -m spacy download en_core_web_lg

# TBD: Import libraries
import spacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

# TBD: Load preferred model

nlp = spacy.load('en_core_web_lg')
# from spacy.lang.en import English
# nlp = English()

# TBD: Load the dataset and test it as-is

with open("electronics_and_cars.txt") as file:
    dataset = file.read()

# print(dataset)

# doc = nlp(dataset)
# print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])



In [3]:
# STEP 1 - TRAIN DATA

# Prepare training data

# TBD: define all the entities by extracting the words and their indexes from the dataset
# expected format is the following:  ("sentence", {"entities": [0,10, "FOOD"]})

words = ['xiaomi', 'samsung', 'nokia', 'apple', 'huawei', 'at&t', 'redmi', 'mediatk', 'oppo', 'motorola', 'oneplus', 'google pixel',
        'tcl', 'lg', 'fiat', 'porsche', 'volkswagen', 'ferrari', 'lotus', 'land rover', 'jaguar', 'citroen', 'ford', 'mercedes',
        'smart', 'jeep', 'suzuki', 'hyundai', 'tesla', 'honda', 'toyota', 'Skoda', 'audi', 'bmw', 'aspark', 'bugatti'
]

# words = ['fiat', 'porsche', 'volkswagen', 'ferrari', 'lotus', 'land rover', 'kia', 'jaguar', 'citroen', 'ford', 'mercedes',
#         'smart', 'jeep', 'suzuki', 'hyundai', 'tesla', 'honda', 'toyota', 'Skoda', 'audi', 'bmw', 'aspark', 'bugatti'
# ]

train_data = []

with open("electronics.txt") as file:
    dataset = file.readlines()
    for sentence in dataset:
        # print("######")
        # print("sentence: ", sentence)
        # print("######")
        sentence = sentence.lower()
        entities = []
        for word in words:
            word = word.lower()
            if word in sentence:
                start_index = sentence.index(word)
                end_index = len(word) + start_index
                # print("word: ", word)
                # print("----------------")
                # print("start index:", start_index)
                # print("end index:", end_index)
                pos = (start_index, end_index, "BRAND")
                entities.append(pos)
        element = (sentence.rstrip('\n'), {"entities": entities})

        train_data.append(element)
        # print('----------------')
        # print("element:", element)

# STEP 2 - UPDATE MODEL

# TBD: load the needed pipeline
ner = nlp.get_pipe("ner")

# TBD: define the annotations
for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# TBD: train the model

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# TBD: define the number of iterations, the batch size and the drop according to your experience or using an empirical value
# Train model

iter = 200
drop_out = 0.3

with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(iter):
        print("Iteration #" + str(iteration))

        # Data shuffle for each iteration
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            for text, annotations in batch:
                # Create an Example object
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=drop_out)
        print("Losses:", losses)

# Save the model
# TBD:

output_dir = Path("./ner/")
nlp.to_disk(output_dir)
print("Saved correctly!")

Iteration #0


/home/alessio/anaconda3/envs/deep_learning/lib/python3.9/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "i've been contemplating on buying a xiaomi smartph..." with entities "[(36, 42, 'ORG'), (92, 97, 'ORG'), (43, 48, 'ORG')...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses: {'ner': 56.56384215070263}
Iteration #1
Losses: {'ner': 35.14746129726168}
Iteration #2
Losses: {'ner': 32.92359875778245}
Iteration #3
Losses: {'ner': 32.89119118715849}
Iteration #4
Losses: {'ner': 33.35417010248601}
Iteration #5
Losses: {'ner': 28.158968211688105}
Iteration #6
Losses: {'ner': 25.095878898729993}
Iteration #7
Losses: {'ner': 28.377173925738806}
Iteration #8
Losses: {'ner': 21.848750470432424}
Iteration #9
Losses: {'ner': 13.010232867805561}
Iteration #10
Losses: {'ner': 14.764382662195006}
Iteration #11
Losses: {'ner': 18.678710184986112}
Iteration #12
Losses: {'ner': 15.087292701882}
Iteration #13
Losses: {'ner': 12.185048794236884}
Iteration #14
Losses: {'ner': 9.48302939684943}
Iteration #15
Losses: {'ner': 9.868819119716669}
Iteration #16
Losses: {'ner': 7.911284909048039}
Iteration #17
Losses: {'ner': 11.60349744011048}
Iteration #18
Losses: {'ner': 10.674368150877976}
Iteration #19
Losses: {'ner': 8.233500852202464}
Iteration #20
Losses: {'ner': 6.63839

In [5]:
# STEP 3 - TEST THE UPDATED MODEL

# Load updated model
print("Loading model...")
nlp_updated = spacy.load(output_dir)

# TBD: test with a old sentence
doc = nlp_updated('Everything you need to know about the Citroën C3 Aircross.')
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])
doc = nlp_updated('Techniques used in Huawei is great, and even awesome.')
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])


# TBD: test with a new sentence and an old brand
doc = nlp_updated('I wll buy the new Tesla car')
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])
doc = nlp_updated('I wll buy the new Motorola smartphone')
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# TBD: test with a new sentence and a new brand
doc = nlp_updated('I find the new Kia car really good')
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])
doc = nlp_updated('I find the new Doogee phone really good')
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])


Loading model...
entities: [('Citroën', 'ORG')]
entities: [('Huawei', 'ORG')]
entities: []
entities: [('Motorola', 'ORG')]
entities: []
entities: []
